Joseph Created this program to tentatively calculate RWA for EQB SFR Corporate entity loans assuming no addon. 
* Step 1: get scored (PD and LGD) EQB SFR data and keep corp entity loans only
* Step 2: force addon to be 0 
* Step 3: Check all relavent columns required for RWA and make assumptions if needed (e.g., maturity =1 given they are all uninsured and it is going to be used in RWA)
* Step 4: Run RWA with addon 0 

In [45]:
#%%
from IPython import get_ipython
get_ipython().magic('reset -sf')

import sys
 
# appending a path
sys.path.append('')
 
import os
import pandas as pd
import numpy as np
import pickle
import SFR_PD_Recalibration_2023_Lib as tool
# import Concentra_SFR_Fit_for_Use_2024_Lib as lib
from matplotlib import pyplot as plt
import math


C:\Users\josepht\AppData\Local\Temp\ipykernel_13932\937880524.py:3: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -sf')


In [46]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [47]:
username = os.getenv('USERNAME')
code_dir= "C:\\Users" +'\\' + username + '\\' + "Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\\code"
input_dir = code_dir + '\\' + "Inputs" #JT
Output_dir = code_dir + '\\' + "Outputs" #JT


snapshot_num = 202412 
snapshot = str(snapshot_num) 
snapshot_date = '2024-12-31' 

#CAR 2023 Ch5 prescribed
correlation_residential_mortgages = 0.15
correlation_residential_mortgages_rental = 0.22

CMHC_pd = 0.0001
CMHC_lgd = 0.11 #according to the newly developed Sovereign LGD model

lgd_gen_floor = 0.1 #general floor by CAR 2023


In [48]:
# get PD Data and chaeck available columns
new_pd_data = pd.read_pickle(input_dir + '\\Rating_July2020_to_Dec2024_EQB (from George 20250402).pkl') # 20250402 George added
new_pd_data_red = new_pd_data.loc[new_pd_data['SL_Date'] == snapshot_date]

# For columns PD_Post_MOC_Pre_Adj and PD_Pre_MOC, if not in the data set, add them with nan #20250402 George added

if 'PD_Post_MOC_Pre_Adj' not in new_pd_data_red.columns: #20250402 George added

    new_pd_data_red['PD_Post_MOC_Pre_Adj'] = np.nan #20250402 George added

if 'PD_Pre_MOC' not in new_pd_data_red.columns: #20250402 George added

    new_pd_data_red['PD_Pre_MOC'] = np.nan #20250402 George added


# Rename AIRB_PD_Risk_Rating to be MRS_Bin #20250402 George added

new_pd_data_red = new_pd_data_red.rename({'AIRB_PD_Risk_Rating':'MRS_Bin'}, axis='columns')  #20250402 George added

C:\Users\josepht\AppData\Local\Temp\ipykernel_13932\536291438.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_pd_data_red['PD_Post_MOC_Pre_Adj'] = np.nan #20250402 George added


In [49]:
#Remove Personal Loans
# print columns in alphabetical order in the new_pd_data_red dataframe #20250402 George added

print(sorted(new_pd_data_red.columns)) #20250402 George added

# Group by MRS_Bin, count distinct Loan_Number and min and max of PD_Post_MOC #20250402 George added
print('***************Before Removing Default Loans*****************') #20250402 George added
print(new_pd_data_red.groupby(['MRS_Bin'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added

# Remove rows with PD_Post_MOC <=0 or >=1 (defaults) #20250402 George added

new_pd_data_red = new_pd_data_red.loc[(new_pd_data_red['Corporate_Applicant_Ind'] == 1) ] 

# Group by MRS_Bin, count distinct Loan_Number and min and max of PD_Post_MOC #20250402 George added

print('***************After Removing Default Loans*****************') #20250402 George added

print(new_pd_data_red.groupby(['MRS_Bin'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added


['Alt_Prime_Indicator', 'AppraisalValue_Inception', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_WOE', 'City', 'Combo_LTV_Insured_Ind', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override', 'Combo_Province_Metro_Override_WOE', 'Corporate_Applicant_Ind', 'Delinquency_Status_Adj', 'Dwelling_Type', 'FSA', 'Funded_Date', 'Insured_Ind', 'LoanType', 'Loan_Number', 'MRS_Bin', 'Metro_Region_BF_FMT', 'PD_Post_MOC', 'PD_Post_MOC_Pre_Adj', 'PD_Pre_MOC', 'PD_scoring', 'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE', 'Province', 'RemainingPrincipal_Excl_Partner', 'RemainingPrincipal_Tot_Exp', 'SL_Date', 'Sub_Product', 'log_odds']
***************Before Removing Default Loans*****************
  MRS_Bin Loan_Number PD_Post_MOC          
              nunique         min       max
0     1.0       26705    0.002228  0.002228
1     2.0       14383    0.003622  0.003622
2     3.0        5505    0.007377  0.007377
3     4.0    

In [50]:
#Remove Default
# print columns in alphabetical order in the new_pd_data_red dataframe 

print(sorted(new_pd_data_red.columns)) 

# Group by MRS_Bin, count distinct Loan_Number and min and max of PD_Post_MOC 
print('***************Before Removing Default Loans*****************') 
print(new_pd_data_red.groupby(['MRS_Bin'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) 

# Remove rows with PD_Post_MOC <=0 or >=1 (defaults) 

new_pd_data_red = new_pd_data_red.loc[(new_pd_data_red['PD_Post_MOC'] > 0) & (new_pd_data_red['PD_Post_MOC'] < 1)] 

# Group by MRS_Bin, count distinct Loan_Number and min and max of PD_Post_MOC 

print('***************After Removing Default Loans*****************') 

print(new_pd_data_red.groupby(['MRS_Bin'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) 

['Alt_Prime_Indicator', 'AppraisalValue_Inception', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_WOE', 'City', 'Combo_LTV_Insured_Ind', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override', 'Combo_Province_Metro_Override_WOE', 'Corporate_Applicant_Ind', 'Delinquency_Status_Adj', 'Dwelling_Type', 'FSA', 'Funded_Date', 'Insured_Ind', 'LoanType', 'Loan_Number', 'MRS_Bin', 'Metro_Region_BF_FMT', 'PD_Post_MOC', 'PD_Post_MOC_Pre_Adj', 'PD_Pre_MOC', 'PD_scoring', 'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE', 'Province', 'RemainingPrincipal_Excl_Partner', 'RemainingPrincipal_Tot_Exp', 'SL_Date', 'Sub_Product', 'log_odds']
***************Before Removing Default Loans*****************
  MRS_Bin Loan_Number PD_Post_MOC          
              nunique         min       max
0     1.0         332    0.002228  0.002228
1     2.0         622    0.003622  0.003622
2     3.0         240    0.007377  0.007377
3     4.0    

In [51]:
new_pd_data_red = new_pd_data_red.rename({'MRS_Bin':'MRS_Bin_PD'}, axis = 'columns')

In [52]:
# get LGD Data and chaeck available columns
new_lgd_data = pd.read_pickle(input_dir + '\\eqb_lgd_scored_2024_12 (from Abhi 20250404).pkl') 
print(sorted(new_lgd_data.columns)) 

['Appraisal_Bin', 'Appraisal_Bin_WOE', 'Appraisal_Bin_WOE', 'BF_Appr_Prov_Dw', 'BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw', 'Default_Ind', 'Final_LGD', 'Foreclosure_Ind', 'Foreclosure_Ind_WOE', 'Insured_Ind', 'LR_Avg_LGD', 'LTV_Bin', 'LTV_Bin_WOE', 'LTV_Bin_WOE', 'LoanType', 'Loan_Number', 'MRS_Bin', 'Mapped_LGD', 'Occupancy', 'Occupancy_WOE', 'Pred_LGD', 'Province_Foreclosure', 'Province_Foreclosure_WOE', 'RemainingPrincipal_Excl_Partner', 'SL_Date', 'Sub_Product']


In [53]:
new_lgd_data.Occupancy.value_counts()

Owner_Occupied    47224
Rental             8405
Name: Occupancy, dtype: int64

In [54]:
if 'LGD_DT_Adjusted' not in new_lgd_data.columns: 
    new_lgd_data['LGD_DT_Adjusted'] = np.nan  

if 'Segment_Avg_LGD' not in new_lgd_data.columns: 
    new_lgd_data['Segment_Avg_LGD'] = new_lgd_data.loc[:,'Mapped_LGD']

if 'Base_Line_LGD' not in new_lgd_data.columns: 
    new_lgd_data['Base_Line_LGD'] = new_lgd_data.loc[:,'Mapped_LGD']

In [57]:
new_lgd_data = new_lgd_data.rename({'MRS_Bin':'MRS_Bin_LGD'}, axis = 'columns')
rwa_raw_data_4 = pd.merge(new_pd_data_red,new_lgd_data[['Loan_Number','MRS_Bin_LGD','LR_Avg_LGD','Segment_Avg_LGD','Base_Line_LGD','LGD_DT_Adjusted','Final_LGD', 'Occupancy']], how='left', on= ['Loan_Number'])
#rwa_raw_data_4 = rwa_raw_data_4.rename({'MRS_Bin':'MRS_Bin_LGD','Final_LGD':'Model_LGD'}, axis = 'columns')
rwa_raw_data_4 = rwa_raw_data_4.rename({'Final_LGD':'Model_LGD'}, axis = 'columns')

In [58]:
print(new_pd_data_red.shape)
rwa_raw_data_4 = rwa_raw_data_4.loc[ ~ (rwa_raw_data_4['Model_LGD'].isna())] #& ( rwa_raw_data_4['Insured_class']== 'Uninsured')
print(rwa_raw_data_4.shape)
rwa_raw_data_4.head()

(1410, 32)
(1410, 39)


,Loan_Number,SL_Date,Delinquency_Status_Adj,RemainingPrincipal_Tot_Exp,Insured_Ind,Dwelling_Type,Funded_Date,FSA,City,Metro_Region_BF_FMT,Province,AppraisalValue_Inception,Prior_24_Worse_Delinquent_Status_FMT_Adj,Corporate_Applicant_Ind,Beacon_Avg_App_CoApp,BF_LTV_Tot_Exp_FSA_Dw_WF,Sub_Product,LoanType,RemainingPrincipal_Excl_Partner,Alt_Prime_Indicator,Combo_Province_Metro_Override,Combo_LTV_Insured_Ind,Beacon_Avg_App_CoApp_WOE,Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE,Combo_Province_Metro_Override_WOE,Combo_LTV_Insured_Ind_WOE,log_odds,PD_scoring,MRS_Bin_PD,PD_Pre_MOC,PD_Post_MOC,PD_Post_MOC_Pre_Adj,MRS_Bin_LGD,LR_Avg_LGD,Segment_Avg_LGD,Base_Line_LGD,LGD_DT_Adjusted,Model_LGD,Occupancy
0,81701,2024-12-31,0 Days,29783.19,Not Insured,Apartment,2003-06-30,M5H,Toronto,Toronto,ON,200000.0,0 Days,1.0,NaN,NaN,EQB,30,29783.19,Alt,ON,,-2.063847,0.471776,0.165650,-0.943851,-3.364749,0.033415,6.0,0.026508,0.034554,NaN,4,0.110595,0.101136,0.101136,NaN,0.5355,Rental
1,84775,2024-12-31,0 Days,32018.74,Not Insured,Apartment,2005-06-30,M5H,Toronto,Toronto,ON,180000.0,0 Days,1.0,NaN,0.045772,EQB,30,32018.74,Alt,ON,"Uninsured, LTV<=0.30",-2.063847,0.471776,0.165650,0.626503,-4.556033,0.010394,4.0,0.008601,0.011499,NaN,1,0.013818,0.010912,0.010912,NaN,0.1074,Rental
2,84787,2024-12-31,0 Days,35482.80,Not Insured,Apartment,2005-06-30,M5H,Toronto,Toronto,ON,190000.0,0 Days,1.0,NaN,0.048054,EQB,30,35482.80,Alt,ON,"Uninsured, LTV<=0.30",-2.063847,0.471776,0.165650,0.626503,-4.556033,0.010394,4.0,0.008601,0.011499,NaN,1,0.013818,0.010912,0.010912,NaN,0.1074,Rental
3,84788,2024-12-31,0 Days,38939.96,Not Insured,Apartment,2005-06-30,M5H,Toronto,Toronto,ON,200000.0,0 Days,1.0,NaN,0.050100,EQB,30,38939.96,Alt,ON,"Uninsured, LTV<=0.30",-2.063847,0.471776,0.165650,0.626503,-4.556033,0.010394,4.0,0.008601,0.011499,NaN,1,0.013818,0.010912,0.010912,NaN,0.1074,Rental
4,87061,2024-12-31,0 Days,732797.16,Not Insured,Single,2006-05-31,T1S,Okotoks,Okotoks,AB,2350000.0,0 Days,1.0,774.0,0.118514,EQB,25,732797.16,Alt,AB_SK_MB_YT_PE_NT_NU,"Uninsured, LTV<=0.30",2.225639,0.471776,-0.389936,0.626503,-6.699171,0.001230,1.0,0.001538,0.002228,NaN,2,0.033370,0.029987,0.029987,NaN,0.1821,Owner_Occupied


In [70]:
rwa_raw_data_4.Occupancy.value_counts()
print(1204/(1204+206))

0.8539007092198582


In [59]:
rwa_raw_data_5 = rwa_raw_data_4.copy()
rwa_raw_data_5['AddOn']=0

if 'LGD_DT_Adjusted' not in rwa_raw_data_5.columns: 
    rwa_raw_data_5['LGD_DT_Adjusted'] = np.nan  # the numbers are not the latest and this will not be used in the final result

rwa_raw_data_5['Advance_Amount'] = rwa_raw_data_5.loc[:,'RemainingPrincipal_Excl_Partner'] # Use this as temporary basis
rwa_raw_data_5['Years_to_maturity'] = 1  #Use this as temporary basis maturity not needed for uninsured loans
rwa_raw_data_5['EAD'] = rwa_raw_data_5.loc[:,'RemainingPrincipal_Excl_Partner'] #Use this as temporary basis
rwa_raw_data_5['Insured_class'] = 'Uninsured'# all corporate entities are uninsured


In [60]:
rwa_raw_data_5['corr_uninsured'] =rwa_raw_data_5['Occupancy'].map(lambda x:0.15 if x in ['owner occupied','Owner Occupied','Owner_Occupied','owner_occupied'] else 0.22)

In [61]:
rwa_raw_data_5['MRS_Bin_PD']=rwa_raw_data_5[]

SyntaxError: invalid syntax (3258863623.py, line 1)

In [62]:
#################################################################
####################### RWA Caculation #########################
#################################################################

df_out=pd.DataFrame()

# record result data 
res_data=pd.DataFrame()


for ind_contol in range(0,3):
    print(ind_contol)

    # baseline LGD, Baseline PD
    if ind_contol==0:
        rwa_raw_data = rwa_raw_data_5.copy().drop(['PD_Post_MOC','PD_Post_MOC_Pre_Adj'], axis=1).rename({'PD_Pre_MOC': 'CalibratedPD', 'Base_Line_LGD' : 'Pre_final_LGD'  }, axis='columns'); 
    #  PD_Post_MOC, LGD_DT_JUST     
    if ind_contol==1:
        rwa_raw_data = rwa_raw_data_5.copy().drop(['PD_Pre_MOC','PD_Post_MOC_Pre_Adj'], axis=1).rename({'PD_Post_MOC': 'CalibratedPD', 'LGD_DT_Adjusted': 'Pre_final_LGD' }, axis='columns'); 
    #  PD_Post_MOC, Final LGD     
    if ind_contol==2:
        rwa_raw_data = rwa_raw_data_5.copy().drop(['PD_Pre_MOC','PD_Post_MOC_Pre_Adj'], axis=1).rename({'PD_Post_MOC': 'CalibratedPD' , 'Model_LGD': 'Pre_final_LGD' }, axis='columns'); 
        
    #  aseline LGD,  PD after long-run adjustment
    '''
    if ind_contol==3:
        rwa_raw_data = rwa_raw_data_5.copy().drop(['PD_Post_MOC','PD_Post_MOC_Pre_Adj'], axis=1).rename({'PD_Pre_MOC': 'CalibratedPD', 'Base_Line_LGD' : 'Pre_final_LGD'  }, axis='columns'); 
        
        adj_Longrun = pd.read_excel(os.path.join(adhoc_path, 'MOC_Summary.xlsx'), sheet_name= 'Sheet1') [['Adj_Longrun']].values[0].item()
        rwa_raw_data['CalibratedPD'] =  rwa_raw_data['CalibratedPD']*(1+adj_Longrun)
    '''
    
    # Apply regulatory floors and derive the final LGD
    rwa_raw_data['DLGD_floor'] = rwa_raw_data['Segment_Avg_LGD'] + rwa_raw_data['AddOn']
    
    # sum(~rwa_raw_data['DLGD_floor'].isna())
    # sum((~rwa_raw_data['Segment_Avg_LGD'].isna()) & (rwa_raw_data.Insured_class.isin(["CMHC","Sagen", "CG"])))
    
    # sum(~rwa_raw_data['Pre_final_LGD'].isna())
    # sum(~rwa_raw_data['Segment_Avg_LGD'].isna() )
    # sum(~(rwa_raw_data.Insured_class.isin(["CMHC","Sagen", "CG"]) ))

    rwa_raw_data.loc[~rwa_raw_data['Pre_final_LGD'].isna(),'OSFI_LGD_floor'] = lgd_gen_floor
    rwa_raw_data['Final_LGD'] = rwa_raw_data[['Pre_final_LGD','DLGD_floor','OSFI_LGD_floor']].max(axis='columns')

    # sum(~rwa_raw_data['Final_LGD'].isna())
    
    #for insured loans which do not have Gen3 LGD, previous Final LGD is used
    #rwa_raw_data.loc[rwa_raw_data['Pre_final_LGD'].isna(),'Final_LGD'] = rwa_raw_data['Final_LGD_old']
    
    
    ## CG and Sagen use 11% LGD 
    rwa_raw_data.loc[rwa_raw_data.Insured_class.isin(["CMHC","Sagen", "CG"]), 'Final_LGD' ] = CMHC_lgd

    # sum(rwa_raw_data['Final_LGD'].isna())
    
    
    ## insured_PMI_ratio 

    
    rwa_raw_data["deductible_amount"] = 0.1 * rwa_raw_data.Advance_Amount
    rwa_raw_data["insured_PMI_ratio"] = np.where(rwa_raw_data.EAD > 0, (rwa_raw_data.EAD - rwa_raw_data.deductible_amount) / rwa_raw_data.EAD, 0)
    
    # !!!! can be negative; When insured_PMI_ratio is negative it will be handle in cond_PMI_dec case.
    
    
  #  sum( round(rwa_raw_data["insured_PMI_ratio"]*10**5) == round(rwa_raw_data["insured_PMI_ratio_old"] *10**5) ) #56478
    
    ## Generate indicators 
    cond_CMHC = (rwa_raw_data.Insured_class.isin(['CMHC']))
    
    cond_PMI = (rwa_raw_data.Insured_class.isin(["Sagen", "CG"])) & (rwa_raw_data.EAD > 0.1 * rwa_raw_data.Advance_Amount)

    cond_PMI_dec = (rwa_raw_data.Insured_class.isin(["Sagen", "CG"])) & (rwa_raw_data.EAD <= 0.1 * rwa_raw_data.Advance_Amount)
    
    cond_UI = (rwa_raw_data.Insured_class.isin(["Uninsured"]))
    
    
    ## Maturity adjustment
    
    def func_maturity_adj(pd_value):
        import numpy as np
        adj = (0.11852 - 0.05478 * np.log(pd_value)) ** 2
        return adj  ##CAR 2023 5.3.1(i) 66.
    
    rwa_raw_data.loc[cond_UI | cond_PMI_dec, "Maturity_adj"] = rwa_raw_data.loc[:, ['CalibratedPD']].apply(lambda s: func_maturity_adj(s[0]), axis=1) #not actually used since retail RWA formula will be applied
    
    rwa_raw_data.loc[cond_CMHC | cond_PMI, "Maturity_adj"] = func_maturity_adj(CMHC_pd)
    

    ## 'corr_uninsured'  depends on rental income indicator and has two values correlation_residential_mortgages_rental,correlation_residential_mortgages
    
    
    # correlation for CMHC and insured deductible for PMI ?
    def correlation_corp(pd_value):
        import numpy as np
        #     corr =  round(0.12 * (1 - np.exp(-50 * pd_value)) / (1- np.exp(-50)) +
        #                       0.24 * (1 - (1- np.exp(-50 * pd_value))/(1 - np.exp(-50))),6) ##CAR 2023 5.3.1(i) 66.
        corr = 0.12 * (1 - np.exp(-50 * pd_value)) / (1 - np.exp(-50)) + 0.24 * (
                1 - (1 - np.exp(-50 * pd_value)) / (1 - np.exp(-50)))
        return corr
    
    rwa_raw_data.loc[cond_CMHC | cond_PMI, 'corr_insured'] = correlation_corp(CMHC_pd)
    
    
    ## func_risk_weight_retail 
    def func_risk_weight_retail(pd_value, lgd_value, corr):  # CAR 2023 Chapter 5 paragrah 81
        import scipy.stats as st
        pd_value = float(pd_value)
        lgd_value = float(lgd_value)
        corr = float(corr)
        pd_UL = st.norm.cdf(
            (1 - corr) ** (-0.5) * st.norm.ppf(pd_value) + (corr / (1 - corr)) ** (0.5) * st.norm.ppf(0.999))
        K = pd_UL * lgd_value - pd_value * lgd_value
        # return  round(pd_UL * lgd_value - pd_value * lgd_value,6)
        return K
    
    
    def func_risk_weight_corp(pd_value, lgd_value, corr, maturity_adj,
                              effective_maturity):  # CAR 2023 Chapter 5 paragraph 66
        import scipy.stats as st
        a = (lgd_value * st.norm.cdf((1 - corr) ** (-0.5) * st.norm.ppf(pd_value) +
                                     (corr / (1 - corr)) ** 0.5 * st.norm.ppf(0.999)))
        tail_value = a - (pd_value * lgd_value)
        adj = (1 + (effective_maturity - 2.5) * maturity_adj) / (1 - (1.5 * maturity_adj))
        k = tail_value * adj
        # return  round(tail_value * adj,6)
        return k
    
    
    # retail risk weight function for uninsured loans
    rwa_raw_data['risk_weight_uninsured'] = 0
    
    rwa_raw_data.loc[cond_UI, 'risk_weight_uninsured'] = rwa_raw_data.loc[cond_UI, ['CalibratedPD', 'Final_LGD', 'corr_uninsured']].apply(
            lambda s: func_risk_weight_retail(s[0], s[1], s[2]), axis=1)
    
    
    # re-write uninsured deductible for PMI using borrower PD and 100% LGD and retail risk weight function
    # df["LGD_uninsured"]=1.0
    
    rwa_raw_data.loc[cond_PMI, 'risk_weight_uninsured'] = rwa_raw_data.loc[cond_PMI, ['CalibratedPD', 'corr_uninsured']].apply(
            lambda s: func_risk_weight_retail(s[0], 1, s[1]), axis=1)
    
    rwa_raw_data.loc[cond_PMI_dec, 'risk_weight_uninsured'] = rwa_raw_data.loc[cond_PMI_dec, ['CalibratedPD', 'corr_uninsured']].apply(
            lambda s: func_risk_weight_retail(s[0], 1, s[1]), axis=1)
    
        
    # insured CMHC including PMI insured backstop use non-retail risk function
    rwa_raw_data['risk_weight_insured'] = 0
    
    rwa_raw_data.loc[cond_CMHC, 'risk_weight_insured'] = rwa_raw_data.loc[cond_CMHC, ['Final_LGD', 'corr_insured','Maturity_adj', 'Years_to_maturity']].apply(
         lambda s: func_risk_weight_corp(CMHC_pd, CMHC_lgd, s[1], s[2], s[3]), axis=1)
    
    
    
    rwa_raw_data.loc[cond_PMI, 'risk_weight_insured'] = rwa_raw_data.loc[cond_PMI, ['Final_LGD', 'corr_insured','Maturity_adj', 'Years_to_maturity']].apply(
         lambda s: func_risk_weight_corp(CMHC_pd,  CMHC_lgd, s[1], s[2], s[3]), axis=1)
    
    
    
    ## RWA_res_mortgage 
    def RWA_res_mortgage(capital_value, capital_value_insured, EAD_value, insured_ratio):
        # insured_ratio: Ratio of insured
        #     insured_RWA = round(capital_value_insured * 12.5 * EAD_value,0)
        #     uninsured_RWA = round(capital_value * 12.5 * EAD_value,0)
        insured_RWA = capital_value_insured * 12.5 * EAD_value
        uninsured_RWA = capital_value * 12.5 * EAD_value
        rwa_total = insured_ratio * insured_RWA + (1 - insured_ratio) * uninsured_RWA
        return rwa_total
    
    
    
    # 1. CMHC insured
    rwa_raw_data.loc[cond_CMHC, 'RWA_new'] = rwa_raw_data.loc[cond_CMHC, [ 'risk_weight_insured', 'EAD']].apply(lambda s: RWA_res_mortgage(0, s[0], s[1], 1), axis=1)
    
    
      
    # 2. Sagen and CG
    rwa_raw_data.loc[cond_PMI, 'RWA_new'] = rwa_raw_data.loc[
          cond_PMI, ['risk_weight_uninsured', 'risk_weight_insured', 'EAD', "insured_PMI_ratio"]].apply(lambda s: RWA_res_mortgage(s[0], s[1], s[2], s[3]), axis=1)

    rwa_raw_data.loc[cond_PMI_dec, 'RWA_new'] = rwa_raw_data.loc[cond_PMI_dec, ['risk_weight_uninsured', 'EAD']].apply(
          lambda s: RWA_res_mortgage(s[0], 0, s[1], 0), axis=1)
    
    # 3. Uninsured Loans
    rwa_raw_data.loc[cond_UI, 'RWA_new'] = rwa_raw_data.loc[cond_UI, ['risk_weight_uninsured', 'EAD']].apply(
          lambda s: RWA_res_mortgage(s[0], 0, s[1], 0), axis=1)
    
    #test = rwa_raw_data.loc[ (round(rwa_raw_data["rwa_airb"]*10**6) != round(rwa_raw_data["RWA_new"]*10**6) )  &  (rwa_raw_data["Insured_class"] == 'CMHC') ][['Insured_class',"rwa_airb",'RWA_new']]
    
    #rwa_raw_data.loc[rwa_raw_data['Insured_class'] =='Uninsufred', 'RWA_new'].sum()
    ## Summary table
    
    if ind_contol==0:
      df_out["EAD"]= [rwa_raw_data["EAD"].sum()]
      # df_out["SA RWA"]= [rwa_raw_data["RWA_standardized"].sum()]  
      # df_out["In Product RWA"]= [rwa_raw_data["rwa_airb"].sum()]  --gen 2 
      df_out["Pre MOC RWA"]= [rwa_raw_data["RWA_new"].sum()]
      
      df_out["Pre MOC RWA (uninsured)"]= [rwa_raw_data.loc[rwa_raw_data['Insured_class']=='Uninsured']["RWA_new"].sum()]
      df_out["EAD (uninsured)"]= [rwa_raw_data.loc[rwa_raw_data['Insured_class']=='Uninsured']["EAD"].sum()]

      
      # rwa_airb_by_Insured_class = rwa_raw_data.groupby(['Insured_class' ],as_index=False)[['EAD','rwa_airb','RWA_standardized']].sum().rename({'rwa_airb': 'rwa_airb_by_Insured_class', 'RWA_standardized': 'RWA_standardized_by_Insured_class'}, axis='columns')
      
      rwa_mapped_by_Insured_class = rwa_raw_data.groupby(['Insured_class' ],as_index=False)[['EAD','RWA_new']].sum().rename({'RWA_new': 'rwa_mapped_by_Insured_class'}, axis='columns')
      
      # rwa_airb_by_MRS_Bin = rwa_raw_data.groupby(['MRS_Bin_PD' ],as_index=False)[['EAD','rwa_airb','RWA_standardized']].sum().rename({'rwa_airb': 'rwa_airb_by_MRS_Bin'}, axis='columns')
      
      rwa_mapped_by_MRS_Bin = rwa_raw_data.groupby(['MRS_Bin_PD' ],as_index=False)[['EAD','RWA_new']].sum().rename({'RWA_new': 'rwa_mapped_by_MRS_Bin'#,'RWA_standardized': 'RWA_standardized_by_MRS_Bin'
                                                                                                           }, axis='columns')
      
      # rwa_by_Insured_class0 =pd.merge(rwa_airb_by_Insured_class, rwa_mapped_by_Insured_class, how='left',on=['Insured_class'] )
      
      # rwa_by_MRS_Bin0 =pd.merge(rwa_airb_by_MRS_Bin, rwa_mapped_by_MRS_Bin, how='left',on=['MRS_Bin_PD'] )
      
    if ind_contol==1: 
        df_out["Post MOC Pre LR RWA"]= [rwa_raw_data["RWA_new"].sum()]
        df_out["Post MOC Pre LR RWA (uninsured)"]= [rwa_raw_data.loc[rwa_raw_data['Insured_class']=='Uninsured']["RWA_new"].sum()]
    
    '''   
    if ind_contol==3: 
        df_out["Post PD longrun"]= [rwa_raw_data["RWA_new"].sum()]
        df_out["Post PD longrun (uninsured)"]= [rwa_raw_data.loc[rwa_raw_data['Insured_class']=='Uninsured']["RWA_new"].sum()]    
    ''' 
        
    if ind_contol==2: 
        df_out["Final RWA"]= [rwa_raw_data["RWA_new"].sum()]

        rwa_Post_MOC_by_Insured_class = rwa_raw_data.groupby(['Insured_class' ],as_index=False)['RWA_new'].sum().rename({'RWA_new': 'rwa_Post_MOC_by_Insured_class'}, axis='columns')

        rwa_Post_MOC_by_MRS_Bin = rwa_raw_data.groupby(['MRS_Bin_PD'],as_index=False)['RWA_new'].sum().rename({'RWA_new': 'rwa_Post_MOC_by_MRS_Bin'}, axis='columns')

        # rwa_by_Insured_class =pd.merge(rwa_by_Insured_class0, rwa_Post_MOC_by_Insured_class, how='left',on=['Insured_class'] )
        rwa_by_Insured_class =pd.merge( rwa_mapped_by_Insured_class, rwa_Post_MOC_by_Insured_class, how='left',on=['Insured_class'] )

        # rwa_by_MRS_Bin =pd.merge(rwa_by_MRS_Bin0, rwa_Post_MOC_by_MRS_Bin, how='left',on=['MRS_Bin_PD'] )
        rwa_by_MRS_Bin =pd.merge(rwa_mapped_by_MRS_Bin, rwa_Post_MOC_by_MRS_Bin, how='left',on=['MRS_Bin_PD'] )

        #copy to res_data
        res_data = rwa_raw_data.copy()

0
1
2


In [63]:
with pd.ExcelWriter(
   Output_dir + '\eqb_corp_rwa_addon_zero.xlsx') as writer:           
     res_data.to_excel(writer, sheet_name="RWA_Data")

In [64]:
rwa_by_Insured_class

,Insured_class,EAD,rwa_mapped_by_Insured_class,rwa_Post_MOC_by_Insured_class
0,Uninsured,798996684.6,9.251038e+07,2.335078e+08


In [65]:
2.335078e+08/798996684.6

0.292251275256418

In [67]:
df_out

,EAD,Pre MOC RWA,Pre MOC RWA (uninsured),EAD (uninsured),Post MOC Pre LR RWA,Post MOC Pre LR RWA (uninsured),Final RWA
0,798996684.6,9.251038e+07,9.251038e+07,798996684.6,4.348574e+07,4.348574e+07,2.335078e+08
